# Multi-threading vs sync, sum example
https://www.educative.io/courses/python-concurrency-for-senior-engineering-interviews/NEm20mBZZy6

In [6]:
from threading import Thread
from multiprocessing import Process
import multiprocessing
import time

In [7]:
class SumUpClass:

    def __init__(self):
        self.counter = 0

    def add_integers(self, start, end):
        for i in range(start, end + 1):
            self.counter += i

    def get_counter(self):
        return self.counter

def single_thread():
    obj = SumUpClass();
    start = time.time();
    obj.add_integers(1, 30000000)
    end = time.time() - start
    print("single threaded took : {} seconds and summed to {}".format(end, obj.counter))

# IMPORTANT, must learn
def multiple_threads():
    obj1 = SumUpClass()
    obj2 = SumUpClass()
    start = time.time()

    t1 = Thread(target=obj1.add_integers, args=(1, 15000000,)) # grammar for thread?
    t2 = Thread(target=obj2.add_integers, args=(15000001, 30000000))

    t1.start()
    t2.start()

    t1.join() # what does join do?
    t2.join()
    combined_sum = obj1.counter + obj2.counter
    end = time.time() - start
    print("multiple threads took : {} seconds and summed to {}".format(end, combined_sum))

In [8]:
print("System has {0} CPUs".format(multiprocessing.cpu_count()))
single_thread()
multiple_threads()

System has 8 CPUs
single threaded took : 3.3199009895324707 seconds and summed to 450000015000000
multiple threads took : 3.3366470336914062 seconds and summed to 450000015000000


# Multi-threading counter example

### Counter, a simple integer counter that does not support any concurrency.

In [12]:
class SingleThreadCounter(object):
    def __init__(self):
        self.value = 0
        
    def increment(self):
        self.value += 1

### FastReadCounter, a counter that is thread-safe and is faster at reading than writing.

In [13]:
# https://julien.danjou.info/atomic-lock-free-counters-in-python/
# locked on write, very slow， query class.value is fast

class FastReadCounter(object):
    def __init__(self):
        self.value = 0
        self._lock = threading.Lock()
        
    def increment(self):
        with self._lock: # equal to acquire + release
            self.value += 1
            '''
            self.lock.acquire()
            self.count += 1
            self.lock.release()
            '''
            
    def value(self):
        with self._lock:
            return self.value

### Fast a counter that is thread-safe and is faster at writing than reading.
There's a way to implement a thread-safe counter in Python that does not need to be locked on write. It's a trick that should only work on CPython because of the Global Interpreter Lock.  
the counter is just incremented without any lock. The GIL protects concurrent access to the internal data structure in C, so there's no need for us to lock anything.   
The value method increments the counter and then gets the value while subtracting the number of times the counter has been read (and therefore incremented for nothing).

In [14]:
import itertools
import threading

class FastWriteCounter(object):
    def __init__(self):
        self._number_of_read = 0
        self._counter = itertools.count()
        self._read_lock = threading.Lock()

    def increment(self):
        next(self._counter)

    def value(self):
        with self._read_lock:
            value = next(self._counter) - self._number_of_read
            self._number_of_read += 1
        return value

In [16]:
import time
N = 1000000

start = time.time()
STC = SingleThreadCounter()
for i in range(N):
    STC.increment()
print("STC write speed: ",time.time()-start)

start = time.time()
FRC = FastReadCounter()
for i in range(N):
    FRC.increment()
print("FRC write speed: ",time.time()-start)

start = time.time()
FWC = FastWriteCounter()
for i in range(N):
    FWC.increment()
print("FWC write speed: ",time.time()-start)

STC write speed:  0.22352218627929688
FRC write speed:  0.4771120548248291
FWC write speed:  0.20882582664489746


# Thread-safe Hashtable

# Practice threading
https://www.educative.io/courses/python-concurrency-for-senior-engineering-interviews/myBBVj1GonR

In [18]:
from threading import Thread
from threading import current_thread


def thread_task(a, b, c, key1, key2):
    print("{0} received the arguments: {1} {2} {3} {4} {5}".format(current_thread().getName(), a, b, c, key1, key2))


myThread = Thread(group=None,  # reserved
                  target=thread_task,  # callable object
                  name="demoThread",  # name of thread
                  args=(1, 2, 3),  # arguments passed to the target
                  kwargs={'key1': 777,
                          'key2': 111},  # dictionary of keyword arguments
                  daemon=None  # set true to make the thread a daemon
                  )

myThread.start()  # start the thread
myThread.join()  # wait for the thread to complete

demoThread received the arguments: 1 2 3 777 111


We can only override the run() method and the constructor of the Thread class.

Thread.__init__() must be invoked if the subclass choses to override the constructor.

Note that the args or kwargs don't get passed to the run method.

In [19]:
class MyTask(Thread):

    def __init__(self):
        # The two args will not get passed to the overridden
        # run method.
        Thread.__init__(self, name="subclassThread", args=(2, 3))

    def run(self):
        print("{0} is executing".format(current_thread().getName()))


myTask = MyTask()

myTask.start()  # start the thread

myTask.join()  # wait for the thread to complete

print("{0} exiting".format(current_thread().getName()))

subclassThread is executing
MainThread exiting


# Daemon thread
The difference between a regular thread and a daemon thread is that a Python program will not exit until all regular/user threads terminate. However, a program may exit if the daemon thread is still not finished.

In [ ]:
The difference between a regular thread and a daemon thread is that a Python program will not exit until all regular/user threads terminate. However, a program may exit if the daemon thread is still not finished.